<a href="https://colab.research.google.com/github/shiling2007/Python-/blob/main/Speedup_python_with_Cython.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# https://colab.research.google.com/drive/1oXcSCor9mH72N8NVeYX4Sl2jD0UWII2n#scrollTo=ooRjCqo0JpW5

from IPython.core.display import display, HTML, Image
display(HTML("<style>.container { width:100% !important; }</style>"))
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"
# from google.colab import drive
# drive.mount('/content/drive')
# from google.colab import files
# files.download('/content/drive/MyDrive/Colab Notebooks/Lease Payment Formula.ipynb') 
import matplotlib.pyplot as plt
import numpy as np
np.set_printoptions(edgeitems=30, linewidth=100000, 
    formatter=dict(float=lambda x: "%.5g" % x))
import pandas as pd
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 999)
pd.set_option("max_colwidth", 500)
# try:
#  device_name = os.environ['COLAB_TPU_ADDR']
#  TPU_ADDRESS = 'grpc://' + device_name
#  print('Found TPU at: {}'.format(TPU_ADDRESS))
# except KeyError:
#  print('TPU not found')
%load_ext autoreload
%autoreload 2

In [ ]:
# Getting started with Cython
# Introduction
# As a big data engineer and machine learning engineer at a multinational reinsurance firm, I frequently have to build data pipelines 
# and these pipelines can often be slow when utilizing standard python. To circumvent clogs in certain memory intensive tasks, I make use of cython
# The Cython language is a superset of Python, this means that almost all python code works in cython. The reason for using cython is 
# that python code can sometimes be slow and by converting some of the slow python code into cython, we can dramatically reduce the 
# runtime. As always, it's best if we begin with an example of the speed benefits of cython.

# To begin, we must load in the cython extension when working inside a jupyter notebook.

In [ ]:
%load_ext cython

In [ ]:
%%cython

# - python version
def py_sum(n):
  """Compute the sum"""
  i = 0
  the_sum = 0
  for i in range(n):
    the_sum += i
  return the_sum

# - cython version
cdef inline int cy_sum(int n):
  cdef int i = 0
  cdef int the_sum = 0
  for i in range(n):
    the_sum += i
  return the_sum

# - cython version bypassing the gil
cdef inline int cy_sum_nogil(int n) nogil:
  cdef int i = 0
  cdef int the_sum = 0
  for i in range(n):
    the_sum += i
  return the_sum

def cy_wrapper(int n):
  return cy_sum(n)

def cy_nogil_wrapper(int n):
  return cy_sum_nogil(n)

In [ ]:
import numpy as np

n=100000000

print("python time complexity")
%timeit py_sum(n)
print('\n')

print('cython time complexity')
%timeit cy_wrapper(n)
print('\n')

print('cython nogil time complexity')
%timeit cy_nogil_wrapper(n)
print('\n')


print("python sum function")
%timeit sum(range(1,n))
print('\n')

print("numpy sum")
%timeit np.sum(range(1, n))

python time complexity
1 loop, best of 5: 4.02 s per loop


cython time complexity
10 loops, best of 5: 76.6 ms per loop


cython nogil time complexity
10 loops, best of 5: 38.5 ms per loop


python sum function
1 loop, best of 5: 2.11 s per loop


numpy sum
1 loop, best of 5: 10.5 s per loop


In [ ]:
cdef int i            # declare an integer variable
cdef double d = 10.1  # declare a double and initialize it to 10.1

In [ ]:
4.02*1000/76.6

52.48041775456919

In [ ]:
%%cython
# add -a to check python iteration


# - declaration of basic C variable types
cdef:
  int i = 0       # integer 
  bint b = True   # boolean
  char c = b'w'   # character
  double d = 10.1 # double
  float f = 1.10  # floating point
  long l = 1000   # long int
  long double ld = 100000000000.10 # long double
  
# - declaration of python types within C
#    these python objects are declared by cython
#    as C pointers to built-in Python struct type
cdef list cy_list
cdef dict cy_dict
cdef str cy_str
cdef set cy_set

In [ ]:
%%cython
cimport cython
from libc.stdlib cimport malloc, realloc, free # import C level functions for memory allocation

# - declare a pointer to a char
cdef char* token = NULL; # good practice to initialize pointers to NULL
# - allocate 5 bytes of memory to token
token = <char*>malloc(5*sizeof(char))

# - insert values into the token array
token[0] = b'h'
token[1] = b'e'
token[2] = b'l'
token[3] = b'l'
token[4] = b'o'
cdef int i = 0
print(chr(token[i]))
print(chr(token[i+1]))
print(chr(token[i+2]))
print(chr(token[i+3]))
print(chr(token[i+4]))
print('\n\n')
# - !WARNING: you must be careful with pointers as you can access memory 
# that may be storing other functions or variables and you can accidentally 
# overwrite them. Use caution when using pointers.

#print(chr(token[i+5])) # we access a memory location not assigned to token

# - if we need more memory, we can use realloc just the C code above
token = <char*>realloc(token, 11*sizeof(char))
token[5] = b' '
token[6] = b'w'
token[7] = b'o'
token[8] = b'r'
token[9] = b'l'
token[10] = b'd'
print(chr(token[i]))
print(chr(token[i+1]))
print(chr(token[i+2]))
print(chr(token[i+3]))
print(chr(token[i+4]))
print(chr(token[i+5]))
print(chr(token[i+6]))
print(chr(token[i+7]))
print(chr(token[i+8]))
print(chr(token[i+9]))
print(chr(token[i+10]))

# - we the C's free function to release the memory taken from the free store(heap)
free(token) # release memory back to the heap
#print(token) # !WARNING: this will cause an error to be raised.

h
e
l
l
o



h
e
l
l
o
 
w
o
r
l
d


In [ ]:
%%cython
import warnings
from libc.stdlib cimport malloc, realloc, free
warnings.filterwarnings(action='once')
# - create a struct called __tok__
cdef struct __tok__:
  char **tokens
  int *num_tokens
  
# - create an enum called week_days
cdef enum week_days:
  Sunday, Monday, Tuesday,
  Wednesday, Thursday,
  Friday, Saturday
  
# - create a union called data
cdef union data:
  double *d_data_array
  int *i_data_array
  char *c_data_array
  float *f_data_array
  
# - create a struct pointer to __tok__ and allocate memory
cdef __tok__ *tPtr = <__tok__*>malloc(sizeof(__tok__))
tPtr.tokens = <char**>malloc(5*sizeof(char*)) # allocate memory for 5 tokens

# - iterate through each token container and allocate memory
cdef int j = 0
for j in range(5):
  tPtr.tokens[j] = <char*>malloc(5*sizeof(char)) ## assign a size of 5 for each token
  
# - load tokens into tPtr.tokens
b1 = b'hello'
b2 = b'world'
b3 = b'there'
b4 = b'other'
b5 = b'nicer'

tPtr.tokens[0] = b1
tPtr.tokens[1] = b2
tPtr.tokens[2] = b3
tPtr.tokens[3] = b4
tPtr.tokens[4] = b5

print(tPtr.tokens[0])
print(tPtr.tokens[1])
print(tPtr.tokens[2])
print(tPtr.tokens[3])
print(tPtr.tokens[4])

b'hello'
b'world'
b'there'
b'other'
b'nicer'


In [ ]:
%%cython

# - fused types
# fused types allow generic programming within cython
# Currently, only variables and function/method arguments can be fuesd types
ctypedef fused int_float_double:
  int
  float
  double

# - ctypedef for the function pointer opr
ctypedef int (*opr)(int, int)

# - same as opr but using fused types
ctypedef int_float_double (*operation)(int_float_double v1, int_float_double v2)

# - compute takes in a function pointer and 2 ints
cdef int compute(opr op, int x, int y):
  return op(x, y)

cdef int add(int a, int b):
  return a + b
cdef int mult(int a, int b):
  return a * b

cdef int_float_double addf(int_float_double a, int_float_double b):
  return a + b
cdef int_float_double multf(int_float_double a, int_float_double b):
  return a * b


# - utilizing the function pointer and ctypedef
print(compute(mult, 10, 20))
print(compute(add, 100, 99))

# - utilizing fused type function pointer and ctypedef
print(addf(10.1, 10))
print(multf(100.1, .512))

200
199
20.1
51.2512


In [ ]:
%%cython -f
# distutils: extra_compile_args = -fopenmp
# distutils: extra_link_args = -fopenmp
# cython: boundscheck = False

# - logic of declarations above
# make sure open mp library is being used

cimport openmp
from cython.parallel import prange, parallel, threadid
from libc.stdlib cimport malloc, realloc, free, abort
from libc.math cimport log

cdef int i
cdef int n = 100000
cdef int sum_ = 0

for i in prange(n, nogil=True): # release the gil
  sum_ += i
print(sum_)

# - function is safe to call from a nogil block
# this does not release the lock
cdef int f(int x) nogil:
  cdef int y
  y = x + 1
  return x + 1

# - function uses the with nogil which releases the lock
cdef int h(int x):
  cdef int y = 0
  cdef int i = 0
  with nogil:
    for i in range(x):
      y += i
  return y
    
def f1(double[:] x, double[:] out):
  cdef int i, n = x.shape[0]
  for i in range(n):
    out[i] = log(x[i])

def f2(double[:] x, double[:] out):
  cdef int i, n = x.shape[0]
  for i in prange(n, nogil=True):
    out[i] = log(x[i])


704982704


In [ ]:
import numpy as np
# - setup data loading
data = np.random.rand(10000000)
out = np.zeros_like(data)

print("f1 time complexity")
%timeit f1(data, out)
print('\n')

print("numpy.log time complexity")
%timeit np.log(data, out)
print('\n')

print("f2 time complexity")
%timeit f2(data, out)
print('\n')

f1 time complexity
1 loop, best of 5: 358 ms per loop


numpy.log time complexity
1 loop, best of 5: 312 ms per loop


f2 time complexity
1 loop, best of 5: 245 ms per loop




In [ ]:
%%cython -f
# distutils: extra_compile_args = -fopenmp
# distutils: extra_link_args = -fopenmp
# cython: boundscheck = False

cimport openmp
from cython.parallel import prange, parallel, threadid
from libc.stdlib cimport malloc, realloc, free, abort
from libc.math cimport log

def f_single(double[:] x):
  cdef int i, n = x.shape[0]
  cdef double result = 0
  for i in range(n):
    if x[i] > 0.5:
      result += log(x[i])
    else:
      result += 1.0
  return result

def f_parallel(double[:] x):
  cdef int i, n = x.shape[0]
  cdef double tmp, result = 0
  for i in prange(n, nogil=True):
    if x[i] > 0.5:
      tmp = log(x[i])
    else:
      tmp = 1.0
    result += tmp
  return result


In [ ]:
import numpy as np
data = np.random.rand(10000000)
out = np.zeros_like(data)

%timeit f_single(data)
%timeit f_parallel(data)

1 loop, best of 5: 201 ms per loop
10 loops, best of 5: 161 ms per loop


In [ ]:
358 / 201 

1.7810945273631842

In [ ]:
from concurrent.futures import ThreadPoolExecutor

def run(f, x, num_threads=4):
  """
  f: function with array parameter
  x: an array
  """
  with ThreadPoolExecutor(max_workers=num_threads) as pool:
    # - sections are memory views(aka pointers) not copies!
    sections = np.array_split(x, num_threads)
    # - create a job for each section of the array
    jobs = [pool.submit(f, s) for s in sections]
  # - wait for each job to finish, return the results
  return sum(j.result() for j in jobs)
    

In [ ]:
%%cython -f
# cython: boundscheck = False
from libc.math cimport log
from libc.stdio cimport *

def f_single(double[:] x):
  cdef int i, n = x.shape[0]
  cdef double result = 0
  for i in range(n):
    if x[i] > 0.5:
      result += log(x[i])
    else:
      result += 1.0
  return result

def f_parallel(double[:] x):
  cdef int i, n = x.shape[0]
  cdef double result = 0
  with nogil:  # <--- this releases the gil!
    for i in range(n):
      if x[i] > 0.5:
        result = log(x[i])
      else:
        result += 1.0
  return result

In [ ]:
print('Direct Calls')
%timeit f_single(data)
%timeit f_parallel(data)
print('\n')
print('using threads')
%timeit run(f_single, data)
%timeit run(f_parallel, data)

Direct Calls
1 loop, best of 5: 203 ms per loop
1 loop, best of 5: 198 ms per loop


using threads
1 loop, best of 5: 204 ms per loop
10 loops, best of 5: 149 ms per loop


In [ ]:
%%cython -f
# distutils: extra_compile_args = -fopenmp
# distutils: extra_link_args = -fopenmp
# cython: boundscheck = False
from libc.math cimport log
from cython.parallel cimport prange

def f_openmp(double[:] x):
  cdef int i, n = x.shape[0]
  cdef double tmp, result = 0
  for i in prange(n, nogil=True):
    if x[i] > 0.5:
      tmp = log(x[i])
    else:
      tmp = 1.0
    result += tmp
  return result

In [ ]:
# - check results
print(f_parallel(data))
print(run(f_parallel, data))
print(f_openmp(data))

%timeit run(f_parallel, data)
%timeit f_openmp(data)
%timeit run(f_parallel, data, num_threads=8)

0.9058745367256471
2.027412683905241
3464268.9774599327
10 loops, best of 5: 148 ms per loop
10 loops, best of 5: 163 ms per loop
10 loops, best of 5: 151 ms per loop
